In [32]:
import time
import json
import pandas as pd

from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

# 1) Start Chrome (ensure chromedriver is on your PATH)
driver = webdriver.ChromiumEdge()

# 2) Load your page
driver.get("http://127.0.0.1:8080/index1.html")
# or: driver.get("file:///full/path/to/NearbySearchJSON.html")





In [ ]:
locs = pd.read_csv("grid.csv")

latitude     40.756162
longitude   -74.018933
Name: 0, dtype: float64

In [ ]:
df_list = []

for idx, row in locs.iterrows():
    latlng = f"{row['latitude']},{row['longitude']}"
    max_retries = 5
    for attempt in range(1, max_retries+1):
        try:
            # 1) enter coords
            inp = driver.find_element(By.ID, "location-input")
            inp.clear()
            inp.send_keys(latlng)

            # 2) click Search
            driver.find_element(By.ID, "search-btn").click()

            # 3) wait for JSON output (starts with “[”)
            WebDriverWait(driver, 1).until(
                lambda d: d.find_element(By.ID, "results")
                            .text.strip()
                            .startswith("[")
            )

            # 4) grab & parse
            results_txt = driver.find_element(By.ID, "results").text
            data = json.loads(results_txt)
            df = pd.DataFrame(data)
            df_list.append(df)
            # success! break out of retry loop
            break

        except (TimeoutException, Exception) as e:
            print(f"Attempt {attempt}/{max_retries} failed for {latlng}: {e!r}")
            if attempt < max_retries:
                time.sleep(1)   # back‑off before retrying
            else:
                print(f"  → Giving up on {latlng} after {max_retries} tries.")
    # end of retry loop

# once done, concatenate all results
all_results = pd.concat(df_list, ignore_index=True)
all_results.to_json("all_results.json",orient="records")
driver.quit()

Attempt 1/5 failed for 40.756162,-74.018933: TimeoutException()
Attempt 2/5 failed for 40.756162,-74.018933: TimeoutException()
Attempt 3/5 failed for 40.756162,-74.018933: TimeoutException()
Attempt 4/5 failed for 40.756162,-74.018933: TimeoutException()
Attempt 5/5 failed for 40.756162,-74.018933: TimeoutException()
  → Giving up on 40.756162,-74.018933 after 5 tries.
Attempt 1/5 failed for 40.756162,-74.01182042807977: StaleElementReferenceException()
Attempt 2/5 failed for 40.756162,-74.01182042807977: TimeoutException()
Attempt 3/5 failed for 40.756162,-74.01182042807977: TimeoutException()
Attempt 4/5 failed for 40.756162,-74.01182042807977: TimeoutException()
Attempt 5/5 failed for 40.756162,-74.01182042807977: TimeoutException()
  → Giving up on 40.756162,-74.01182042807977 after 5 tries.
Attempt 1/5 failed for 40.756162,-74.00115157019943: StaleElementReferenceException()
Attempt 2/5 failed for 40.756162,-74.00115157019943: TimeoutException()
Attempt 3/5 failed for 40.756162,-

In [37]:
import pandas as pd

In [39]:
df = pd.read_json("lowermanhattan.json")

In [61]:
df.shape

(768, 4)

In [49]:
df[['lat','lng']] = pd.json_normalize(df['loc'])

In [58]:
df = df.drop(columns="loc")

In [57]:
df.head()

,name,loc,status,lat,lng
0,Bluestone Lane 55 Hudson Yards Café,"{'lat': 40.7551684, 'lng': -74.0015389}",OPERATIONAL,40.755168,-74.001539
1,Starbucks,"{'lat': 40.7547724, 'lng': -74.0019329}",OPERATIONAL,40.754772,-74.001933
2,Starbucks,"{'lat': 40.757815, 'lng': -74.002465}",OPERATIONAL,40.757815,-74.002465
3,Bluestone Lane 55 Hudson Yards Café,"{'lat': 40.7551684, 'lng': -74.0015389}",OPERATIONAL,40.755168,-74.001539
4,Starbucks,"{'lat': 40.7547724, 'lng': -74.0019329}",OPERATIONAL,40.754772,-74.001933


In [60]:
df.drop_duplicates(inplace=True)

In [62]:
df.to_json("cafes.json",orient="records")